# FFR120 Simulation of Complex Systems - Crowd Disasters
**Group** - Team Butterflies

**Members**:
- Artur ..
- Maria
- Nina
- Ruiqi
- Rundong .,

In [ ]:
from dataclasses import dataclass
import numpy as np


@dataclass(init=True)
class SimState:
    positions: np.ndarray
    velocities: np.ndarray
    person_states: np.ndarray
        
        
class Environment:    
    # A rectangle is defined as (topleft, topright, botright, botleft)
    
    # Obstacles are defined as a set of points,
    # thus the total shape becomes (n_obstacles, n_corners, n_dim)
    # example: 3 rectangular obstacles -> shape=(3, 4, 2)
    obstacles: np.ndarray = np.array([
        
    ])
        
    # Exits are defined as rectangular,
    # and thus behave in the same way as obstacles
    exits: np.ndarray = np.array([
        [[40, 24], [51, 24], [51, 26], [40, 26]]
    ])
        
    
class PersonState:
    living = 0
    exited = 1
    dead = 2
        
    
class SimConstants:
    time_inc = 0.01
    n_individuals = 300
    individual_radius = 0.5
    collision_rebound = 100
    mass = 10
    max_pos = 50, 50
    simulation_time = 10
    damping_constant = 10
    
    n_time_steps = int(simulation_time/time_inc)
    force_scalar = time_inc/mass
    
        
        
class AnimationSettings:
    display_size = 8, 8   # Size of display in inches
    dpi = 100    # Pixels per inch
    bg_color = "#C336C9"
    individual_color = "#000000"
    framerate = 20
    exit_color = "#00FF00"
    
    def get_marker_size():
        return AnimationSettings.dpi*np.mean(AnimationSettings.display_size)*SimConstants.individual_radius/np.mean(SimConstants.max_pos)

## Simulation


In [ ]:
def get_environmental_force(individual, positions, velocities, forces) -> float:
    """Net forces acting on an individual caused by the environment"""
    
    # Compute the displacement between this individual and all others, |p_i - p_j|.
    displacements = positions[individual, :] - positions

    # The distance to each individual is the norm of each displacement |p_i - p_j|.
    distances = np.linalg.norm(displacements, axis=1)

    # Find all the collisions that occur, which is defined as the instances
    # where the distances are within 2 radiuses, since this means that each
    # perimeter is just barely touching.
    # Exclude the instances where the distance is 0, since this means that
    # the individual is being compared to itself (or something has gone terribly wrong):
    collisions = (
        (distances < 2*SimConstants.individual_radius) &
        (distances > 0)
    )

    net_environmental_force = np.sum(SimConstants.collision_rebound / displacements[collisions], axis=0)
    
    return net_environmental_force

In [ ]:
def get_social_force(individual, positions, velocities, forces) -> float:
    """Net forces acting on an individual caused by the environment"""

    velocity = velocities[individual]
    damping_force = -np.sign(velocity) * (velocity*velocity.T) * SimConstants.damping_constant
    
    
    
    net_social_force = damping_force
    
    return net_social_force

In [ ]:
def get_overlap_individuals_with_rectangle(positions, rectangle):
    # First check if bounding box of individual overlaps with the rectangle
    bbox_botleft = positions - SimConstants.individual_radius
    bbox_topright = positions + SimConstants.individual_radius
    
    i_rect_overlap = np.where(
        (bbox_botleft[:, 0] > rectangle[0, 0]) &
        (bbox_topright[:, 0] < rectangle[2, 0]) &
        (bbox_botleft[:, 1] > rectangle[0, 1]) &
        (bbox_topright[:, 1] < rectangle[2, 1])
    )
    
    # TODO:
    # Now see if circle overlaps
    # https://www.baeldung.com/cs/circle-line-segment-collision-detection
    
    return i_rect_overlap
    

In [ ]:
import numpy as np

def run_simulation(seed: int = None) -> list[SimState]:   
    np.random.seed(seed)

    max_pos = np.array(SimConstants.max_pos)
    zero_vectors = np.zeros(shape=(SimConstants.n_individuals, 2))
    positions = np.random.rand(SimConstants.n_individuals, 2)
    positions *= max_pos
    velocities = np.zeros(shape=(SimConstants.n_individuals, 2))
    person_states = np.zeros(shape=(SimConstants.n_individuals,))

    state = SimState(positions=positions, velocities=velocities, person_states=person_states)
    history = [state]
    
    for time_step in range(SimConstants.n_time_steps):
        active = person_states == PersonState.living
        
        positions[active] = state.positions[active] + state.velocities[active] * SimConstants.time_inc
        forces = np.zeros(shape=positions.shape)

        for individual in range(SimConstants.n_individuals): 
            if not active[individual]:
                continue
                
            forces[individual] += get_environmental_force(individual, positions, velocities, forces)
            forces[individual] += get_social_force(individual, positions, velocities, forces)

        velocities[active] = (
            state.velocities[active] +
            forces[active] * SimConstants.force_scalar
        )

        state = SimState(positions=positions.copy(), velocities=velocities.copy(), person_states=person_states.copy())
        history.append(state)
        
        i_exiting = get_overlap_individuals_with_rectangle(positions, Environment.exits[0])

    return history


In [ ]:
history_temp = run_simulation()


## Animation

In [ ]:
%matplotlib notebook

import matplotlib.pyplot as plt
import matplotlib.animation as anim
import matplotlib.patches as patches


def get_time_step(i_frame: int, frame_rate: int, time_inc: float) -> int:
    """Get the time step corresponding to actual time passed of simulation.

    Example:
        50 fps -> 0.02s per frame
        0.01 time_inc -> 2 time steps per frame
        3 frames -> 6 time_steps

    Args:
        i_frame: The frame currently being rendered:
        frame_rate: Number of frames rendered per second.
        time_inc: The time passed between time steps.
    """
    return int((i_frame / frame_rate) / time_inc)


def get_rectangle_pos_and_size_for_patch(rectangles, index) -> tuple[np.ndarray, float, float]:
    """Extract position, width, and height from the rectangle set.
    
    Each rectangle is defined by the points (topleft, topright, botright, botleft).
    A patch is defined by the (botleft, width, height).
    """
    
    rect = rectangles[index]
    pos = rect[3, :]
    w = rect[2, 0]-rect[0, 0]
    h = rect[2, 1]-rect[0, 1]
    return pos, w, h 


def render_state(i_frame, history, screen):
    i_time = get_time_step(i_frame, AnimationSettings.framerate, SimConstants.time_inc)
    state = history[i_time]
    screen.set_data(state.positions[:, 0], state.positions[:, 1])
                
    return screen, 


def render_simulation(history: list[SimState]) -> None:
    fig = plt.figure(
        figsize=AnimationSettings.display_size,
        dpi=AnimationSettings.dpi
    )
    ax = fig.gca()
    screen, = ax.plot([], [], 'o', ms=AnimationSettings.get_marker_size())

    exits = []
    for i_exit in range(len(Environment.exits)):
        exit = patches.Rectangle(
            *get_rectangle_pos_and_size_for_patch(Environment.exits, i_exit),
            color=AnimationSettings.exit_color
        )
        ax.add_patch(exit)
        exits.append(exit)
    
    ax.set_xlim(0, SimConstants.max_pos[1])
    ax.set_ylim(0, SimConstants.max_pos[0])
        
    
    animation = anim.FuncAnimation(
        fig=fig, func=render_state, frames=len(history),
        fargs=(history, screen),
        blit=True, interval=int(1000/AnimationSettings.framerate)
    )
    return animation
    
animation = render_simulation(history_temp)
